In [2]:
import torch
# from processing_image import Preprocess
# from modeling_frcnn import GeneralizedRCNN
# from utils import Config
# from transformers import LxmertTokenizer, LxmertForPreTraining
import os, json
from PIL import Image
import random
from tqdm import tqdm
import open_clip
import numpy as np
from training.data import NpyDataset
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
checkpoint="/home/mila/l/le.zhang/scratch/open_clip/src/Outputs/negclip/checkpoints/epoch_0.pt"
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model, _, image_preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained="openai",device=device)
model, _, image_preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=checkpoint, device=device)
tokenizer = open_clip.get_tokenizer('ViT-B-32')



In [3]:
eval_dataset_path="/home/mila/l/le.zhang/scratch/winonground/data/processed_dataset/coco_val"
eval_dataset=NpyDataset(eval_dataset_path,image_preprocess,1000,tokenizer)
eval_dataloader=DataLoader(eval_dataset,batch_size=256,shuffle=False,num_workers=0)
len(eval_dataset)


merging 5 splied files from /home/mila/l/le.zhang/scratch/winonground/data/processed_dataset/coco_val


1000

In [41]:
gt_caption_features=[]
hn_caption_features=[]
all_image_features=[]
for i in tqdm(eval_dataloader):
    gt_caption=i[1][:,0,:].to(device)
    hn_caption=i[1][:,1::].reshape(-1,77).to(device)
    
    images=i[0].to(device)
    model.eval()
    with torch.no_grad(), torch.cuda.amp.autocast():
        
        image_features = model.encode_image(images)
        gt_text_features = model.encode_text(gt_caption)
        hn_text_features = model.encode_text(hn_caption)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        gt_text_features /= gt_text_features.norm(dim=-1, keepdim=True)
        hn_text_features /= hn_text_features.norm(dim=-1, keepdim=True)
        gt_caption_features.append(gt_text_features)
        hn_caption_features.append(hn_text_features)
        all_image_features.append(image_features)
all_caption_features=torch.cat((torch.cat(gt_caption_features),torch.cat(hn_caption_features)))
all_image_features=torch.cat(all_image_features)

100%|██████████| 4/4 [00:11<00:00,  2.92s/it]


In [14]:
features=torch.load("/home/mila/l/le.zhang/scratch/open_clip/src/itr_results/saved_features/rank_coco-dis_text_mean-hn--5e-06-weightd0.5-weightr0.2-ub5-w_special_epoch_1_features.pt")
caption_features=features["caption_features"].detach().cpu().numpy()
image_features=features["image_features"].detach().cpu().numpy()

In [15]:
# write to file
np.savez_compressed("/home/mila/l/le.zhang/scratch/open_clip/src/itr_results/saved_features/rank_coco-dis_text_mean-hn--5e-06-weightd0.5-weightr0.2-ub5-w_special_epoch_1_features.npz",caption_features=caption_features,image_features=image_features)

In [7]:
def compute_similarity(image_features, text_features, bs = 500):
    # compute similarity
    dim0 = image_features.shape[0]
    dim1 = text_features.shape[0]
    similarity_scores = torch.zeros(dim0, dim1)
    for v in range(0, dim0, bs):
        for t in range(0, dim1, bs):
            print('Processing Visual '+str(v)+' Text '+str(t), end='\r')
            batch_visual_emb = image_features[v:v+bs].cuda()
            batch_caption_emb = text_features[t:t+bs].cuda()
            with torch.no_grad():
                logits = batch_visual_emb @ batch_caption_emb.t()
                logits.cpu()caption_features
            similarity_scores[v:v+bs,t:t+bs] = logits
            # clear torch cuda cache
            torch.cuda.empty_cache()
            del batch_visual_emb, batch_caption_emb, logits


    print('Done similarity')
    return similarity_scores
compute_similarity(image_features,caption_features)

: 

: 

In [44]:
metrics = {}

logits_per_image = (model.logit_scale * all_image_features @ all_caption_features.t()).detach().cpu()
logits = {"image_to_text": logits_per_image}
ground_truth = torch.arange(len(all_image_features)).view(-1, 1)

for name, logit in logits.items():
    ranking = torch.argsort(logit, descending=True)
    preds = torch.where(ranking == ground_truth)[1]
    preds = preds.detach().cpu().numpy()
    metrics[f"{name}_mean_rank"] = preds.mean() + 1
    metrics[f"{name}_median_rank"] = np.floor(np.median(preds)) + 1
    for k in [1, 5, 10]:
        metrics[f"{name}_R@{k}"] = np.mean(preds < k)


In [3]:
import torch
import numpy as np
clip_features=torch.load("/home/mila/l/le.zhang/scratch/open_clip/src/itr_results/saved_features/openai_pretrainedtest_features.pt")


# test time to calculate cosine similarity between a and b
import time
start=time.time()

metrics = {}
image_features=clip_features['image_features']
text_features=clip_features['caption_features']
logits_per_image = compute_similarity(image_features,text_features)

logits = {"image_to_text": logits_per_image}
ground_truth = torch.arange(len(image_features)).view(-1, 1)

for name, logit in logits.items():
    ranking = torch.argsort(logit, descending=True)
    preds = torch.where(ranking == ground_truth)[1]
    preds = preds.detach().cpu().numpy()
    metrics[f"{name}_mean_rank"] = preds.mean() + 1
    metrics[f"{name}_median_rank"] = np.floor(np.median(preds)) + 1
    for k in [1, 5, 10]:
        metrics[f"{name}_R@{k}"] = np.mean(preds < k)
end=time.time()
print(end-start)


Done similarityal 9500 Text 49500


: 

: 

In [8]:
# read from file
import torch
import numpy as np
features=torch.load("/home/mila/l/le.zhang/scratch/open_clip/src/itr_results/saved_features/rank_coco-dis_text_mean-hn--5e-06-weightd0.5-weightr0.2-ub5-w_special_epoch_1_features.pt")
text_features=features["caption_features"][:50000]
image_features=features["image_features"][:10000]
print(text_features.shape)
print(image_features.shape)

torch.Size([50000, 512])
torch.Size([10000, 512])


In [5]:
subset_num=50000
text_features=torch.cat(text_features[:subset_num],text_features[image_features.shape[0]:image_features.shape[0]+subset_num*4])
image_features=image_features[:subset_num]



In [9]:
from tqdm import tqdm
def compute_retrieval(a2b_sims, return_ranks=True ,start_index=0):
    """
    Args:
        a2b_sims: Result of computing similarity between two sets of embeddings (emb1 @ emb2.T)
            with shape (num_datapoints, num_datapoints).

    Returns:
        Retrieval metrics for that similarity.
    """
    npts = a2b_sims.shape[0]
    ranks = torch.zeros(npts)
    # loop source embedding indices
    for index in tqdm(range(start_index,npts+start_index)):
        ranking = torch.argsort(a2b_sims, descending=True)
        preds = torch.where(ranking == index)[1]
        preds = preds.detach().cpu()
      
        ranks[index] = preds
    return ranks

bs=1000
split_num=50
image_len = image_features.shape[0]
text_len  = text_features.shape[0]
split_size=image_len//split_num
total_ranks=[]
for eid,partial_index in tqdm(enumerate(range(0, image_len, split_size))):
    partial_image_features = image_features[partial_index:partial_index+split_size]
    max_pairs=partial_image_features.shape[0]
    sub_similarity_scores = torch.zeros((max_pairs, text_len))
    for v in range(0, max_pairs, bs):
        for t in range(0, text_len, bs):
            batch_visual_emb = partial_image_features[v:v+bs]
            batch_caption_emb = text_features[t:t+bs]
            with torch.no_grad():
                logits = batch_visual_emb @ batch_caption_emb.t()
            sub_similarity_scores[v:v+bs,t:t+bs] = logits.detach().cpu()
    a2b_sims=sub_similarity_scores.detach().cpu()
    start_index=eid*split_size
    npts = a2b_sims.shape[0]

    # loop source embedding indices
    # for index in tqdm(range(start_index,npts+start_index)):
    #     ranking = torch.argsort(a2b_sims, descending=True)
    #     preds = torch.where(ranking == index)[1]
    #     preds = preds.detach().cpu()
    #     ranks.append(preds)

    ground_truth=torch.arange(eid*split_size,min((eid+1)*split_size,image_len)).reshape(-1,1)
    ranking = torch.argsort(a2b_sims, descending=True) 
    preds = torch.where(ranking == ground_truth)[1]
    # rank,top1 = compute_retrieval(sub_similarity_scores.detach().cpu(),eid*split_size)
    total_ranks.append(preds)
metrics={}
total_ranks=torch.cat(total_ranks)
def mean_of_list(l):
    return round((sum(l)/len(l)).item(),2)
name="clip"
for k in [1, 5, 10]:
    metrics[f"{name}_R@{k}"] = mean_of_list(total_ranks < k)
metrics[f"{name}_mean_rank"] = round(total_ranks.mean(dtype=torch.float32).item())+ 1
metrics[f"{name}_median_rank"] = np.floor(np.median(total_ranks)) + 1

50it [00:20,  2.49it/s]


In [11]:
metrics

{'clip_R@1': 0.06,
 'clip_R@5': 0.19,
 'clip_R@10': 0.26,
 'clip_mean_rank': 440,
 'clip_median_rank': 57.0}